## Web server interface at https://xxxx:7473

#### Update - since the videos were filmed, neo4j requires a longer, more complex password, so the newest password is here:

**Username: neo4j**

**Password: ucb_mids_w205**

The above web server allows and interactive GUI which can output graphs visually in addition to table like output.  The nodes in the graphs can be moved around with the mouse to make the graphs more readable.


#### Basics:

```:server connect``` - connect to the server, username is "neo4j", password is "ucb_mids_w205"


```:server status``` - shows that username and server you are logged into


```:clear``` - clears off old cells


```show databases``` - note that community edition only has 1 application database that we can use neo4j, we cannot create now use other databases, we have to wipe out neo4j database for each new graph


## In the Neo4j GUI, run the following query with graph output and rearrange the nodes with your mouse if necessary:

```match (n) return n```

In [1]:
import neo4j

import pandas as pd

from IPython.display import display

## Connect, login, create driver, create session; with community edition, we can only use 1 database, the "neo4j" database


In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

## my_neo4j_wipe_out_database() - since community edition can only have 1 database "neo4j", this function will wipe out all the nodes and relationships

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

## my_neo4j_run_query_pandas() will run a Cypher query and put the results in a Pandas dataframe; easy to see how you can use Python to manipulate the returned data

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

## my_neo4j_nodes_relationships() will print the nodes (assumes a name property) and relationships

In [6]:
def my_neo4j_nodes_relationships():
    "print all the nodes and relationships"
   
    print("-------------------------")
    print("  Nodes:")
    print("-------------------------")
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    display(df)
    
    print("-------------------------")
    print("  Relationships:")
    print("-------------------------")
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    display(df)
    
    density = (2 * number_relationships) / (number_nodes * (number_nodes - 1))
    
    print("-------------------------")
    print("  Density:", f'{density:.1f}')
    print("-------------------------")
    

## Load CSV

In [7]:
df = pd.read_csv("NASDAQ_100_Data_From_2010.csv", sep='\t', parse_dates=["Date"])


In [8]:
df['return'] = df.groupby("Name")['Close'].pct_change()

# Our ne4j doesn't have pearson correlation so we comput pearson correlation in pandas
# the pearson correlations are based on the pct change of the prices over the dates
returns_pivot = df.pivot(index='Date', columns='Name', values='return')
correlations = returns_pivot.corr().fillna(0)

In [9]:
correlations

Name,AAPL,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,AMGN,...,TSLA,TXN,VRSK,VRSN,VRTX,WBA,WDAY,XEL,XLNX,ZM
Name,,,,,,,,,,,,,,,,,,,,,
AAPL,1.000000,0.488606,0.503710,0.464148,0.461381,0.260991,0.356629,0.503264,0.357316,0.369982,...,0.313554,0.527643,0.376615,0.433669,0.254777,0.288543,0.382561,0.285327,0.424430,0.169929
ADBE,0.488606,1.000000,0.544967,0.536524,0.634392,0.280554,0.405513,0.527878,0.377873,0.418025,...,0.346311,0.550980,0.449556,0.500540,0.297366,0.294064,0.571972,0.321669,0.493240,0.288472
ADI,0.503710,0.544967,1.000000,0.560382,0.552071,0.259753,0.430096,0.719527,0.459223,0.428041,...,0.335483,0.811336,0.427458,0.478004,0.283664,0.329651,0.428815,0.304108,0.692629,0.067026
ADP,0.464148,0.536524,0.560382,1.000000,0.516684,0.460661,0.437561,0.535603,0.332356,0.494094,...,0.275272,0.589917,0.556937,0.509739,0.300558,0.411425,0.401131,0.494208,0.467863,-0.044615
ADSK,0.461381,0.634392,0.552071,0.516684,1.000000,0.244344,0.424727,0.548294,0.394638,0.364310,...,0.328712,0.565297,0.390734,0.489180,0.285664,0.309458,0.526177,0.260982,0.497439,0.222399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBA,0.288543,0.294064,0.329651,0.411425,0.309458,0.309931,0.246176,0.334522,0.185585,0.373512,...,0.154363,0.357009,0.295288,0.261524,0.197165,1.000000,0.188806,0.300267,0.295239,-0.141406
WDAY,0.382561,0.571972,0.428815,0.401131,0.526177,0.128264,0.350667,0.422774,0.307401,0.312557,...,0.333367,0.424635,0.375046,0.423419,0.280678,0.188806,1.000000,0.152384,0.381218,0.243818
XEL,0.285327,0.321669,0.304108,0.494208,0.260982,0.835311,0.246843,0.280353,0.176345,0.366112,...,0.115936,0.316649,0.429160,0.315936,0.182841,0.300267,0.152384,1.000000,0.220571,-0.039379


## Load graph into neo4j

In [10]:
companies = ['CREATE']

# Loop through the tickers to create nodes
for ticker in correlations.columns:
    companies.append(f'(:Company {{ticker: "{ticker}"}})')
    
companies_query = ", ".join(companies[1:])

companies_query = f"CREATE {companies_query}"

In [11]:
my_neo4j_wipe_out_database()

query = companies_query

my_neo4j_run_query_pandas(query)

""


## Create relationships using correlation data

In [12]:
for i in correlations.columns:
    for j in correlations.columns:
        # just not add weaker correlations to graph
        if i != j and correlations.loc[i, j] > 0.5:
            relationships_query = f"""
                                MATCH (a:Company {{ticker: '{i}'}}), (b:Company {{ticker: '{j}'}})
                                MERGE (a)-[:CORRELATED_WITH {{correlation: {correlations.loc[i, j]}}}]->(b)
                                """   
            my_neo4j_run_query_pandas(relationships_query)
            

In [13]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
my_neo4j_run_query_pandas(query)


,graphName


In [14]:
query = """
CALL gds.graph.project(
  'ds_graph',
  'Company',
  {
    CORRELATED_WITH: {
      orientation: 'UNDIRECTED',
      properties: ['correlation']
    }
  }
)
"""
my_neo4j_run_query_pandas(query)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Company': {'label': 'Company', 'properties':...",{'CORRELATED_WITH': {'orientation': 'UNDIRECTE...,ds_graph,102,2008,1600


In [15]:
query = """
CALL gds.leiden.write('ds_graph', {
  writeProperty: 'community',
  includeIntermediateCommunities: false
})
YIELD communityCount, modularity
RETURN communityCount, modularity
"""
result = my_neo4j_run_query_pandas(query)

In [16]:
query = """
MATCH (c:Company)
RETURN c.ticker AS ticker, c.community AS community
ORDER BY community, ticker
"""
leiden_df = my_neo4j_run_query_pandas(query)

In [17]:
display(leiden_df)

,ticker,community
0,PTON,0
1,REGN,1
2,DLTR,2
3,ATVI,3
4,EA,3
...,...,...
97,MRNA,33
98,ORLY,34
99,LULU,35
100,JD,36


In [18]:
# join sector to dataframe
company_info = pd.read_csv('nasdaq_screener.csv')
company_info = company_info.rename(columns={'Symbol': 'ticker'})

leiden_df = pd.merge(
    leiden_df, 
    company_info[['ticker', 'Name', 'Sector']], 
    on='ticker', 
    how='left'
)

In [19]:
pd.set_option('display.max_rows', None)
display(leiden_df)

,ticker,community,Name,Sector
0,PTON,0,Peloton Interactive Inc. Class A Common Stock,Consumer Discretionary
1,REGN,1,Regeneron Pharmaceuticals Inc. Common Stock,Health Care
2,DLTR,2,Dollar Tree Inc. Common Stock,Consumer Discretionary
3,ATVI,3,NaN,NaN
4,EA,3,Electronic Arts Inc. Common Stock,Technology
5,EBAY,4,eBay Inc. Common Stock,Consumer Discretionary
6,DXCM,6,DexCom Inc. Common Stock,Health Care
7,AAPL,7,Apple Inc. Common Stock,Technology
8,ADBE,7,Adobe Inc. Common Stock,Technology
9,ADSK,7,Autodesk Inc. Common Stock,Technology


## Visualizations for Community Detection

In [20]:
# import necessary packages for visualization
%pip install nx_altair altair

     |████████████████████████████████| 731 kB 24.3 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 5.8 MB/s s eta 0:00:01
     |████████████████████████████████| 325 kB 93.8 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
Note: you may need to restart the kernel to use updated packages.


In [21]:
import altair as alt
import networkx as nx
import nx_altair as nxa
from neo4j import GraphDatabase

In [22]:
query = "MATCH (c:Company) RETURN c.ticker AS ticker, c.community AS community"
result = session.run(query)
communities_df = pd.DataFrame([r.data() for r in result])

In [69]:
# bar chart plots
community_counts = communities_df['community'].value_counts().reset_index()
community_counts.columns = ['community', 'count']

chart = alt.Chart(community_counts).mark_bar().encode(
    x=alt.X('community:N', title='Community'),
    y=alt.Y('count:Q', title='Number of Stocks'),
    tooltip=['community', 'count']
).properties(
    title='Stock Count per Louvain Community',
    width=600,
    height=400
)

chart

alt.Chart(...)

In [24]:
# table of communities and their stocks
grouped = communities_df.groupby('community')['ticker'].apply(lambda tickers: ', '.join(tickers[:5]))

grouped_df = grouped.reset_index()
grouped_df.columns = ['Community', 'Stocks']
grouped_df

,Community,Stocks
0,0,PTON
1,1,REGN
2,2,DLTR
3,3,"ATVI, EA"
4,4,EBAY
5,6,DXCM
6,7,"AAPL, ADBE, ADSK, AMZN, ANSS"
7,8,PDD
8,9,WBA
9,10,SGEN


In [30]:
company_info.head()

,ticker,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$99.29,-4.3000,-4.151%,2.830785e+10,United States,1999.0,4525196,Industrials,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$22.57,-2.0200,-8.215%,5.843019e+09,United States,2016.0,8923904,Industrials,Aluminum
2,AACB,Artius II Acquisition Inc. Class A Ordinary Sh...,$9.95,0.0700,0.709%,0.000000e+00,United States,2025.0,144,NaN,NaN
3,AACBU,Artius II Acquisition Inc. Units,$10.081,-0.0190,-0.188%,0.000000e+00,United States,2025.0,828,Finance,Blank Checks
4,AACG,ATA Creativity Global American Depositary Shares,$0.93,0.0115,1.252%,2.976012e+07,China,2008.0,6493,Real Estate,Other Consumer Services


In [27]:
corr_matrix = correlations.copy()

#remove names of index/column to avoid key error
corr_matrix.index.name = None
corr_matrix.columns.name = None

# remove self-correlations
for col in corr_matrix.columns:
    corr_matrix.loc[col, col] = pd.NA

corr_long = corr_matrix.stack().reset_index()
corr_long.columns = ['Stock1', 'Stock2', 'Correlation']

# drop duplicate pairs
corr_long['pair'] = corr_long.apply(lambda row: tuple(sorted([row['Stock1'], row['Stock2']])), axis=1)
corr_long = corr_long.drop_duplicates(subset='pair').drop(columns='pair')

In [48]:
top_20 = 20
top_corr = corr_long.sort_values(by='Correlation', ascending=False).head(20)
top_corr['label'] = top_corr['Stock1'] + ' — ' + top_corr['Stock2']
top_corr = (
    top_corr
    .merge(company_info.rename(columns={'ticker': 'Stock1', 'sector': 'Sector1'}), on='Stock1', how='left')
    .merge(company_info.rename(columns={'ticker': 'Stock2', 'sector': 'Sector2'}), on='Stock2', how='left')
)

In [64]:
# FISV is missing sector and name, manually add it
top_corr.loc[top_corr['Stock2'] == 'FISV', 'Name_y'] = 'Fiserv'
top_corr.loc[top_corr['Stock2'] == 'FISV', 'Sector_y'] = 'Technology'

top_corr['SectorCombo'] = top_corr.apply(
    lambda row: f"{row['Sector_x']} — {row['Sector_y']}", axis=1
)

chart = alt.Chart(top_corr).mark_bar().encode(
    x=alt.X('Correlation:Q', title='Correlation'),
    y=alt.Y('label:N', sort='-x', title='Stock Pair'),
    color=alt.Color('SectorCombo:N', title='Sector Pair', scale=alt.Scale(scheme='set2')),
    tooltip=['Name_x', 'Sector_x', 'Name_y', 'Sector_y', 'Correlation']
).properties(
    width=600,
    height=400,
    title=f'Top 20 Most Correlated Stock Pairs From 2010-2021'
)

chart

alt.Chart(...)

## Page Rank

In [72]:
# reload the graph database
my_neo4j_wipe_out_database()

query = companies_query

my_neo4j_run_query_pandas(query)

""


In [73]:
for i in correlations.columns:
    for j in correlations.columns:
        # just not add weaker correlations to graph
        if i != j and correlations.loc[i, j] > 0.5:
            relationships_query = f"""
                                MATCH (a:Company {{ticker: '{i}'}}), (b:Company {{ticker: '{j}'}})
                                MERGE (a)-[:CORRELATED_WITH {{correlation: {correlations.loc[i, j]}}}]->(b)
                                """   
            my_neo4j_run_query_pandas(relationships_query)
            

In [74]:
query = "CALL gds.graph.drop('ds_graph', false) yield graphName"
my_neo4j_run_query_pandas(query)


,graphName
0,ds_graph


In [75]:

query = """
CALL gds.graph.project(
    'ds_graph',
    'Company',
    'CORRELATED_WITH',
    {
        relationshipProperties: ['correlation']
    }
)
"""
my_neo4j_run_query_pandas(query)



,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Company': {'label': 'Company', 'properties':...","{'CORRELATED_WITH': {'orientation': 'NATURAL',...",ds_graph,102,1004,15


In [76]:
# run pagerank
query = """
CALL gds.pageRank.stream('ds_graph',
                        { 
                          maxIterations: $max_iterations,
                          dampingFactor: $damping_factor,
                          relationshipWeightProperty: 'correlation',
                          relationshipTypes: ['CORRELATED_WITH']
                        })
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).ticker AS ticker, score as page_rank
ORDER BY page_rank DESC, ticker ASC
"""
max_iterations = 100
damping_factor = 0.85

pagerank_results = my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor)

In [77]:
display(pagerank_results)

,ticker,page_rank
0,PYPL,2.302784
1,FISV,2.253226
2,HON,2.243895
3,ADP,2.169328
4,PAYX,2.168148
5,TXN,2.164597
6,ADI,2.093625
7,SNPS,2.044410
8,MCHP,2.011714
9,INTU,1.894881


In [78]:
# join sector to dataframe
company_info = pd.read_csv('nasdaq_screener.csv')
company_info = company_info.rename(columns={'Symbol': 'ticker'})

pagerank_df = pd.merge(
    pagerank_results, 
    company_info[['ticker', 'Name', 'Sector']], 
    on='ticker', 
    how='left'
)

In [79]:
pd.set_option('display.max_rows', None)
display(pagerank_df)

,ticker,page_rank,Name,Sector
0,PYPL,2.302784,PayPal Holdings Inc. Common Stock,Consumer Discretionary
1,FISV,2.253226,NaN,NaN
2,HON,2.243895,Honeywell International Inc. Common Stock,Industrials
3,ADP,2.169328,Automatic Data Processing Inc. Common Stock,Technology
4,PAYX,2.168148,Paychex Inc. Common Stock,Consumer Discretionary
5,TXN,2.164597,Texas Instruments Incorporated Common Stock,Technology
6,ADI,2.093625,Analog Devices Inc. Common Stock,Technology
7,SNPS,2.044410,Synopsys Inc. Common Stock,Technology
8,MCHP,2.011714,Microchip Technology Incorporated Common Stock,Technology
9,INTU,1.894881,Intuit Inc. Common Stock,Technology


In [92]:
pagerank_df.loc[pagerank_df['ticker'] == 'FISV', 'Name'] = 'Fiserv'
pagerank_df.loc[pagerank_df['ticker'] == 'FISV', 'Sector'] = 'Technology'

top_pr = pagerank_df.sort_values(by='page_rank', ascending=False).head(25)

alt.Chart(top_pr).mark_bar().encode(
    x=alt.X('page_rank:Q', title='PageRank Score'),
    y=alt.Y('Name:N', sort='-x', title='Stock'),
    color=alt.Color('Sector:N', scale=alt.Scale(scheme='tableau20')),
    tooltip=['ticker', 'Name', 'page_rank', 'Sector']
).properties(
    title='Top 25 Most Influential Stocks (PageRank)',
    width=600,
    height=500
)

alt.Chart(...)

In [94]:
pagerank_df.loc[pagerank_df['ticker'] == 'FISV', 'Name'] = 'Fiserv'
pagerank_df.loc[pagerank_df['ticker'] == 'FISV', 'Sector'] = 'Technology'

top_pr = pagerank_df.sort_values(by='page_rank', ascending=False).head(25)

alt.Chart(top_pr).mark_bar().encode(
    x=alt.X('page_rank:Q', title='PageRank Score'),
    y=alt.Y('ticker:N', sort='-x', title='Stock'),
    color=alt.Color('Sector:N', scale=alt.Scale(scheme='tableau20')),
    tooltip=['ticker', 'Name', 'page_rank', 'Sector']
).properties(
    title='Top 25 Most Influential Stocks (PageRank)',
    width=600,
    height=500
)

alt.Chart(...)